Using SQLAlchemy to Talk to a Database
=====================
SqlAlchemy helps you use a database to store and retrieve information from python.  It abstracts the specific storage engine from te way you use it - so it doesn't care if you end up using MySQL, SQLite, or whatever else. In addition, you can use core and the object-relational mapper (ORM) to avoid writing any SQL at all.  The [SQLAlchemy homepage](http://www.sqlalchemy.org/) has lots of good examples and full documentation.

In [1]:
from sqlalchemy import *
import datetime
import mediacloud
MyList = ['Clinton','Trump','robot','kitten','happy']
mc = mediacloud.api.MediaCloud('03c4af1a8327ec7ee8e305d3e52720a672feacee9894bc9136215d5a544b31ff')    

## Basic SQL Generation
The core library generates SQL for you.  Read more about it on their [expression language tutorial page](http://docs.sqlalchemy.org/en/rel_1_0/core/index.html). Below are some basic examples.

### Creating a Table
Read more about [defining and creating tables](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#define-and-create-tables).

In [2]:
# add `echo=True` to see log statements of all the SQL that is generated
engine = create_engine('sqlite:///:memory:',echo=True) # just save the db in memory for now (ie. not on disk)
metadata = MetaData()
# define a table to use
queries = Table('queries', metadata,
    Column('id', Integer, primary_key=True),
    Column('keywords', String(400), nullable=False),
    Column('count',Integer),
    Column('timestamp', DateTime, default=datetime.datetime.now),
)
metadata.create_all(engine) # and create the tables in the database

2016-12-03 00:37:59,755 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2016-12-03 00:37:59,759 INFO sqlalchemy.engine.base.Engine ()
2016-12-03 00:37:59,763 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2016-12-03 00:37:59,771 INFO sqlalchemy.engine.base.Engine ()
2016-12-03 00:37:59,775 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("queries")
2016-12-03 00:37:59,787 INFO sqlalchemy.engine.base.Engine ()
2016-12-03 00:37:59,795 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE queries (
	id INTEGER NOT NULL, 
	keywords VARCHAR(400) NOT NULL, 
	count INTEGER, 
	timestamp DATETIME, 
	PRIMARY KEY (id)
)


2016-12-03 00:37:59,860 INFO sqlalchemy.engine.base.Engine ()
2016-12-03 00:37:59,864 INFO sqlalchemy.engine.base.Engine COMMIT


### Inserting Data
Read more about generating [SQL insert statements](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#insert-expressions).

In [3]:
#insert_stmt = queries.insert()
#str(insert_stmt) # see an example of what this will do

In [4]:
for item in MyList:
    res = mc.sentenceCount(item, solr_filter=[mc.publish_date_query( datetime.date( 2016, 9, 1), datetime.date( 2016, 9, 30) ), 'tags_id_media:1' ])
    r = res['count']
    insert_stmt = queries.insert().values(keywords= item, count = r)
    db_conn = engine.connect()
    result = db_conn.execute(insert_stmt)
    result.inserted_primary_key # print out the primary key it was assigned
    #str(insert_stmt)

2016-12-03 00:38:01,756 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, count, timestamp) VALUES (?, ?, ?)
2016-12-03 00:38:01,765 INFO sqlalchemy.engine.base.Engine ('Clinton', 133808, '2016-12-03 00:38:01.755906')
2016-12-03 00:38:01,781 INFO sqlalchemy.engine.base.Engine COMMIT
2016-12-03 00:38:02,673 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, count, timestamp) VALUES (?, ?, ?)
2016-12-03 00:38:02,677 INFO sqlalchemy.engine.base.Engine ('Trump', 198155, '2016-12-03 00:38:02.673549')
2016-12-03 00:38:02,681 INFO sqlalchemy.engine.base.Engine COMMIT
2016-12-03 00:38:03,389 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, count, timestamp) VALUES (?, ?, ?)
2016-12-03 00:38:03,389 INFO sqlalchemy.engine.base.Engine ('robot', 3196, '2016-12-03 00:38:03.389383')
2016-12-03 00:38:03,389 INFO sqlalchemy.engine.base.Engine COMMIT
2016-12-03 00:38:04,205 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, count, timestamp

In [5]:
#insert_stmt = queries.insert().values(keywords="kittens")
#result = db_conn.execute(insert_stmt)
#result.inserted_primary_key # print out the primary key it was assigned

### Retrieving Data
Read more about using [SQL select statments](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#selecting).

In [6]:
from sqlalchemy.sql import select
select_stmt = select([queries])
results = db_conn.execute(select_stmt)
sum = 0
for row in results:
    sum  = sum + row[2]
print "Total count:"
print sum

2016-12-03 00:38:05,280 INFO sqlalchemy.engine.base.Engine SELECT queries.id, queries.keywords, queries.count, queries.timestamp 
FROM queries
2016-12-03 00:38:05,288 INFO sqlalchemy.engine.base.Engine ()
Total count: 354327


In [7]:
#select_stmt = select([queries]).where(queries.c.id==1)
#for row in db_conn.execute(select_stmt):
    #print(row)

In [8]:
#select_stmt = select([queries]).where(queries.c.keywords.like('p%'))
#for row in db_conn.execute(select_stmt):
    #print(row)

## ORM
You can use their ORM library to handle the translation into full-fledged python objects.  This can help you build the Model for you [MVC](https://en.wikipedia.org/wiki/Model–view–controller) solution.

In [9]:
#import datetime
#from sqlalchemy import *
#from sqlalchemy.ext.declarative import declarative_base


### Creating a class mapping
Read more about [creating a mapping](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#declare-a-mapping).

In [10]:
# class Query(Base):
#     __tablename__ = 'queries'
#     id = Column(Integer, primary_key=True)
#     keywords = Column(String(400))
#     timestamp = Column(DateTime,default=datetime.datetime.now)
#     def __repr__(self):
#         return "<Query(keywords='%s')>" % (self.keywords)
# Query.__table__

### Creating a connection and session
Read more about [creating this stuff](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#creating-a-session).

In [11]:
# engine = create_engine('sqlite:///:memory:') # just save the db in memory for now (ie. not on disk)
# Base.metadata.create_all(engine)
# Session = sessionmaker(bind=engine)
# my_session = Session()

### Inserting Data
Read more about [inserting data with an ORM](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#adding-new-objects).

In [12]:
# query = Query(keywords="iguana")
# query.keywords

In [13]:
# my_session.add(query)
# my_session.commit()
# query.id

### Retrieving Data
Read more about [retrieving data from the db](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#querying) via an ORM class.

In [14]:
#for q in my_session.query(Query).order_by(Query.timestamp):
#    print(q)

In [15]:
#query1 = Query(keywords="robot")
#query2 = Query(keywords="puppy")
#my_session.add_all([query1,query2])
#my_session.commit()

In [16]:
#for q in my_session.query(Query).order_by(Query.timestamp):
#    print(q)

In [17]:
#for q in my_session.query(Query).filter(Query.keywords.like('r%')):
#    print(q)